# Synthetic Control Analysis
このノートブックでは、Base チェーンのトランザクション数に対する合成制御分析を実行します。

In [28]:
#必要なライブラリのインポート
from datetime import datetime
import json
import pandas as pd
import sys
import os

# scriptsディレクトリをパスに追加\
sys.path.append(os.path.join(os.getcwd(), '..'))
from scripts.counterfactual import create_synth_control, SynthControlRequest
from scripts.visualization import make_linechart

## データの読み込み
GrowthePieのデータを読み込みます。

In [29]:
# データの読み込み
with open('../data/_local/fundamentals_full.json', 'r') as f:
		gtp_data = json.load(f)
df_gtp = pd.DataFrame(gtp_data)
print(df_gtp)

            metric_key origin_key        date         value
0       market_cap_usd   ethereum  2023-04-07  2.255496e+11
1       market_cap_usd   ethereum  2023-04-08  2.248847e+11
2       market_cap_usd   ethereum  2023-04-09  2.231692e+11
3       market_cap_usd   ethereum  2023-04-10  2.242195e+11
4       market_cap_usd   ethereum  2023-04-11  2.302017e+11
...                ...        ...         ...           ...
366121         txcount      rhino  2025-01-03  6.493000e+03
366122       aa_last7d      rhino  2025-01-03  1.196500e+04
366123         txcount   starknet  2025-01-03  5.897300e+04
366124             daa   starknet  2025-01-03  5.873000e+03
366125       aa_last7d   starknet  2025-01-03  2.665000e+04

[366126 rows x 4 columns]


## 分析パラメータの設定
Synthetic Control分析のパラメータを設定します。

In [33]:
request = SynthControlRequest(
    time_predictors_prior_start=datetime(2023, 1, 1),  # 期間を短く
    time_predictors_prior_end=datetime(2023, 12, 31),
    time_optimize_ssr_start=datetime(2023, 8, 1),
    time_optimize_ssr_end=datetime(2023, 12, 31),
    dependent='txcount',
    treatment_identifier='base',
    controls_identifier=[
        'arbitrum', 'optimism', 'zksync_era',
        'polygon_zkevm', 'scroll', 'mantle'
    ],  # 制御群を減らす
    predictors=[
        'txcount',
        'tvl',
        'txcosts_median_eth'
    ]  # 予測変数を減らす
)

## 分析の実行
設定したパラメータを使用して分析を実行します。

In [34]:
# 分析の実行
response = create_synth_control(df_gtp, request)

# 結果の確認
print("Control weights:")
print(json.dumps(response.weights, indent=2))

/Users/tanakashuuhei/developer/synthetic_control_project/notebooks/../scripts/counterfactual.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  treatment_values = treatment_data[train_mask][
/Users/tanakashuuhei/developer/synthetic_control_project/notebooks/../scripts/counterfactual.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  treatment_values = treatment_data[train_mask][
/Users/tanakashuuhei/developer/synthetic_control_project/notebooks/../scripts/counterfactual.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  control_values = control_data[train_mask][
/Users/tanakashuuhei/developer/synthetic_control_project/notebooks/../scripts/counterfactual.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  control_values = control_data[train_mask][
/Users/tanakashuuhei/developer/synthetic_control_project/notebooks/../scripts/counterfactual.py:49: UserWarn

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (18,) + inhomogeneous part.

## 結果の可視化
分析結果をグラフで表示します。

In [26]:
# データの可視化
df = pd.DataFrame(response.data).melt(id_vars='date', var_name='cohort')
fig = make_linechart(
df=df[df['date']>'2023-08-01'],
    title="Transactions on Base vs Ethereum and other L2s",
    treatment_label="Treatment (Base)",
    events=[
    {'date': '2024-03-13', 'text': 'Dencun upgrade'},
    ]
    )
fig.show()

NameError: name 'response' is not defined